# DIP By HanXuan

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import time

import torch
from torch.autograd import Variable

from net.hourglass2 import hourglassNetwork
from util.recorder import experimentalRecord
from util.visual import *

In [ ]:
# standard deviation of added noise after each training set
sigma = 1./30
# number of training iterations
num_steps = 5001
# number of steps to take before saving an output image
save_frequency = 500
# choose either 'ps' or 'dc' as the architecture used.
net_file = 'net/architecture/dmitry5_skip543.json'
# input image. the architectures have been designed for 512x512 colour images
image_path = 'images/image_Baboon512rgb_noi_s20.png'
# where to put the output
expName = 'image_Baboon512rgb_noi_s20_dmitry5_skip543_inputVib'
expDiscribe = 'just as name'

In [ ]:
record = experimentalRecord(basePath='../DIP_result/denoising',
                            expName=expName,
                            author='HanXuan',
                            describe=expDiscribe,
                            fileName='Experimental Record.txt')
record.open()

In [ ]:
# import image
deconstructed = jpg_to_tensor(image_path)
deconstructed = Variable(deconstructed)
image_np = deconstructed.cpu().data.numpy()
plot_image(image_np,(8,8))
record.add_image(image=image_np,
                imageName='origin.png',
                message='Original deconstructed image.',
                mode='NP')

In [ ]:
# input of the network is noise
noise = Variable(torch.randn(deconstructed.data.shape).cuda())

#initialise the network with the chosen architecture
net = hourglassNetwork(net_file)
    
#bind the network to the gpu if cuda is enabled

net.cuda()
start_time = time.time()
#network optimizer set up
learning_rate = lambda step: 1e-2 if (step<=2000) else 1e-3
counter = 0
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate(counter))
#dummy index to provide names to output files
save_img_ind = 0
for step in range(num_steps):
    # get the network output
    output = net(noise)
    # calculate the l2_loss over the masked output and take an optimizer step
    optimizer.zero_grad()
    loss = torch.sum((output - deconstructed)**2)
    loss.backward()
    optimizer.step()
    counter += 1
    #every save_frequency steps, save a jpg
    if step % save_frequency == 0:
        time_cost = time.time() - start_time
        loss_value = loss.cpu().data.numpy()[-1]
        # Cost time 236.978221s  At step 02000  loss is 28938.4121094
        print_message='Cost time %fs'%(time_cost) + '  At step %05d  loss is %f' %(step, loss_value)
        image_np = output.cpu().data.numpy()
        plot_image(image_np,(10,10))
        record.add_image(image=image_np,
                         imageName=str(save_img_ind)+'.png',
                         message=print_message,
                         mode='NP')        
        print(print_message)
        save_img_ind += 1
        noise.data += sigma * torch.randn([1,3,512,512]).cuda()


In [ ]:
record.close()